In [8]:
import numpy as np
in_files = ("alarm.bif", "records.dat")

In [9]:
def read_records(filename = "records.dat"):
    # read records from file
    with open(filename, 'r') as f:
        records = f.readlines()

    # convert records to numpy array of columns
    records = np.array([record.split() for record in records]).T

    return records

records = read_records(in_files[1])
records

array([['"False"', '"False"', '"False"', ..., '"False"', '"False"',
        '"False"'],
       ['"Normal"', '"Normal"', '"Normal"', ..., '"High"', '"Normal"',
        '"Normal"'],
       ['"False"', '"?"', '"False"', ..., '"False"', '"False"',
        '"False"'],
       ...,
       ['"Normal"', '"Normal"', '"Normal"', ..., '"Low"', '"Normal"',
        '"Normal"'],
       ['"Normal"', '"Normal"', '"Normal"', ..., '"Normal"', '"Normal"',
        '"Normal"'],
       ['"Normal"', '"Normal"', '"Normal"', ..., '"OneSided"',
        '"Normal"', '"Normal"']], dtype='<U12')

In [10]:
variable_mapping = {}
def parse_bif_variables(file_path):
    """
    Parses the .bif file to extract variable names and their possible values.
    
    Args:
    - file_path (str): Path to the .bif file.
    
    Returns:
    - dict: A dictionary where keys are variable names and values are lists of possible values.
    """
    global variable_mapping
    variables = {}
    with open(file_path, "r") as file:
        lines = file.readlines()
        
    # Flags to determine if we are within a variable block
    inside_variable_block = False
    current_variable = None
    
    counter = 0
    for line in lines:
        line = line.strip()  # Remove leading and trailing whitespace
        if line.startswith("variable"):
            # Extract variable name
            current_variable = line.split("\"")[1]
            
            variable_mapping[current_variable] = counter
            counter += 1
            
            inside_variable_block = True
        elif inside_variable_block and line.startswith("type discrete"):
            # Extract possible values for the variable
            values = line.split("{")[1].split("}")[0].split()
            values = [v.strip() for v in values]
            variables[current_variable] = values
        elif inside_variable_block and line == "}":
            # End of the variable block
            inside_variable_block = False
            current_variable = None
            
    return variables

variables = parse_bif_variables(in_files[0])
variables, variable_mapping

({'Hypovolemia': ['"True"', '"False"'],
  'StrokeVolume': ['"Low"', '"Normal"', '"High"'],
  'LVFailure': ['"True"', '"False"'],
  'LVEDVolume': ['"Low"', '"Normal"', '"High"'],
  'PCWP': ['"Low"', '"Normal"', '"High"'],
  'CVP': ['"Low"', '"Normal"', '"High"'],
  'History': ['"True"', '"False"'],
  'MinVolSet': ['"Low"', '"Normal"', '"High"'],
  'VentMach': ['"Zero"', '"Low"', '"Normal"', '"High"'],
  'Disconnect': ['"True"', '"False"'],
  'VentTube': ['"Zero"', '"Low"', '"Normal"', '"High"'],
  'KinkedTube': ['"True"', '"False"'],
  'Press': ['"Zero"', '"Low"', '"Normal"', '"High"'],
  'ErrLowOutput': ['"True"', '"False"'],
  'HRBP': ['"Low"', '"Normal"', '"High"'],
  'ErrCauter': ['"True"', '"False"'],
  'HREKG': ['"Low"', '"Normal"', '"High"'],
  'HRSat': ['"Low"', '"Normal"', '"High"'],
  'BP': ['"Low"', '"Normal"', '"High"'],
  'CO': ['"Low"', '"Normal"', '"High"'],
  'HR': ['"Low"', '"Normal"', '"High"'],
  'TPR': ['"Low"', '"Normal"', '"High"'],
  'Anaphylaxis': ['"True"', '"Fa

In [11]:
records_weight_column = np.array([1.0]*len(records[0]))
probs = {}
def get_prob(var_names, permutation):
    count = np.array([True]*len(records[0]))
    
    for i in range(len(var_names)):
        count = count & (records[variable_mapping[var_names[i]]] == permutation[i])
    prob = np.sum(records_weight_column[count])/np.sum(records_weight_column)
    
    probs[str(var_names) + str(permutation)] = prob
    
    return prob

In [12]:
import itertools

def process_probablity_line(line):
    print(line)
    extract = line.split('-1')
    var_names = [i for i in extract[0].replace(" ", '')
                  .split('(')[1].split(')')[0].split('"') if len(i)!=0]
    permutations = list(itertools.product(*[variables[var_name] for var_name in var_names]))
    print(var_names, permutations)
    prob = [get_prob(var_names, permutation) for permutation in permutations]
    print(prob)
    
    return extract[0] + " ".join([str(i) for i in prob]) + extract[-1]

In [13]:
def update_bif(in_filename=in_files[0], out_filename=f"solved_{in_files[0]}"):
    with open(in_filename, 'r') as f_in:
        with open(out_filename, 'w') as f_out:
            while True:
                line = f_in.readline()
                if not line:
                    break
                if "probability" in line:
                    while '}' not in line:
                        line += f_in.readline()
                    line = process_probablity_line(line)
                f_out.write(line)

In [14]:
update_bif()

probability (  "Hypovolemia" ) { //1 variable(s) and 2 values
	table -1 -1 ;
}

['Hypovolemia'] [('"True"',), ('"False"',)]
[0.1982882882882883, 0.7761261261261261]
probability (  "StrokeVolume"  "LVFailure"  "Hypovolemia" ) { //3 variable(s) and 12 values
	table -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ;
}

['StrokeVolume', 'LVFailure', 'Hypovolemia'] [('"Low"', '"True"', '"True"'), ('"Low"', '"True"', '"False"'), ('"Low"', '"False"', '"True"'), ('"Low"', '"False"', '"False"'), ('"Normal"', '"True"', '"True"'), ('"Normal"', '"True"', '"False"'), ('"Normal"', '"False"', '"True"'), ('"Normal"', '"False"', '"False"'), ('"High"', '"True"', '"True"'), ('"High"', '"True"', '"False"'), ('"High"', '"False"', '"True"'), ('"High"', '"False"', '"False"')]
[0.00990990990990991, 0.018018018018018018, 0.16756756756756758, 0.03504504504504505, 0.0, 0.01756756756756757, 0.0075675675675675675, 0.6265765765765766, 0.0, 0.0006306306306306306, 0.002072072072072072, 0.0336036036036036]
probability (  "LVFailur